In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
url="https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=2022;trophy=117;type=season"

Requests library is used to get the data through url.
BeautifulSoup is used to parse through the content received from the requests object. It hepls to view the data in organized form as shown in the cell no.4 so that we can understand the content and plan accordingly.
Pandas library provides as data structure called DataFrame in which we can store the data like we store in a table, and also we can save the data into a csv file.

In [23]:
html=requests.get(url)

In [24]:
soup=BeautifulSoup(html.content,"html.parser")

In [25]:
table=soup.table
table

<table class="engineTable">
<caption>Match results</caption>
<thead>
<tr class="head">
<th class="left" nowrap="nowrap" title="team">Team 1</th>
<th class="left" nowrap="nowrap" title="team">Team 2</th>
<th class="left" nowrap="nowrap" title="winner/result">Winner</th>
<th class="left" nowrap="nowrap" title="result margin">Margin</th>
<th class="left" nowrap="nowrap" title="ground played on">Ground</th>
<th nowrap="nowrap" title="match date span">Match Date</th>
<th nowrap="nowrap" title="match scorecard">Scorecard</th>
</tr>
</thead>
<tbody>
<tr class="data1" data-days="738605">
<td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/335974.html">Super Kings</a></td>
<td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/335971.html">KKR</a></td>
<td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/335971.html">KKR</a></td>
<td class="left" nowrap="nowrap">6 wickets</td>
<td class="left"><a class="data-link" href="/ci/

In [26]:
titles=[]
for i in table.thead.tr.find_all('th'):
    titles.append(i.text)
titles[-1]="Match Link"
titles

['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'Match Link']

Above are the headings of the match details table.

In [27]:
data=pd.DataFrame(columns=titles)

Creating a DataFrame using the above headings

In [28]:
table.tbody.tr.find_all('td')[-1].a['href']

'/ci/engine/match/1304047.html'

With this link we can access the score card of each match

In [29]:
for i in table.tbody.find_all('tr'):
    j=i.find_all('td')
    team1=j[0].text
    team2=j[1].text
    winner=j[2].text
    margin=j[3].text
    ground=j[4].text
    match_date=j[5].text
    match_link=j[6].a['href']
    match_id=match_link.split("/")[-1].split('.')[0]
    data=data.append({'Match ID':match_id,'Team 1':team1,'Team 2':team2,'Winner':winner,'Margin':margin,
                      'Ground':ground,'Match Date':match_date,'Match Link':match_link},ignore_index=True)

data

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match Link,Match ID
0,Super Kings,KKR,KKR,6 wickets,Wankhede,"Mar 26, 2022",/ci/engine/match/1304047.html,1304047
1,Capitals,Mumbai,Capitals,4 wickets,Brabourne,"Mar 27, 2022",/ci/engine/match/1304048.html,1304048
2,Punjab Kings,RCB,Punjab Kings,5 wickets,DY Patil,"Mar 27, 2022",/ci/engine/match/1304049.html,1304049
3,Titans,Super Giants,Titans,5 wickets,Wankhede,"Mar 28, 2022",/ci/engine/match/1304050.html,1304050
4,Sunrisers,Royals,Royals,61 runs,Pune,"Mar 29, 2022",/ci/engine/match/1304051.html,1304051
...,...,...,...,...,...,...,...,...
69,Sunrisers,Punjab Kings,Punjab Kings,5 wickets,Wankhede,"May 22, 2022",/ci/engine/match/1304116.html,1304116
70,Titans,Royals,Titans,7 wickets,Eden Gardens,"May 24, 2022",/ci/engine/match/1312197.html,1312197
71,Super Giants,RCB,RCB,14 runs,Eden Gardens,"May 25, 2022",/ci/engine/match/1312198.html,1312198
72,Royals,RCB,Royals,7 wickets,Ahmedabad,"May 27, 2022",/ci/engine/match/1312199.html,1312199


In [30]:
data.to_csv('Match_Results.csv')

In [34]:
html2=requests.get("https://stats.espncricinfo.com"+table.tbody.tr.find_all('td')[-1].a['href'])

In [35]:
soup2=BeautifulSoup(html2.content,'html.parser')

In [36]:
batting_titles=[]
bowling_titles=[]
for i in soup2.find_all('table')[0].thead.tr.find_all('th'):
    batting_titles.append(i.text)
for i in soup2.find_all('table')[1].thead.tr.find_all('th'):
    bowling_titles.append(i.text)

In [37]:
batting_titles

['BATTING', '\xa0', 'R', 'B', 'M', '4s', '6s', 'SR']

In [38]:
bowling_titles

['BOWLING', 'O', 'M', 'R', 'W', 'ECON', '0s', '4s', '6s', 'WD', 'NB']

In [39]:
batting_data=pd.DataFrame(columns=["Match ID","Match",'Team',"Position",'Batter Name','Player ID','Out','Runs','Balls','4s','6s','Strike Rate'])
bowling_data=pd.DataFrame(columns=["Match ID","Match",'Team','Bowler Name','Player ID','Overs','Made INs','Runs','Wickets','Economy Rate','Dots','4s','6s','Wides','No Balls'])

In [40]:
for i in soup2.find_all('table')[1].tbody.tr.find_all('td'):
    print(i.text)

Umesh Yadav
4
0
20
2
5.00
15
1
1
2
1


In [41]:
for i in soup2.find_all('div'):
    try:
        if i.span.span.span.text=="Match Details":
            print(i.parent.div.span.span.span.text)
            print(i.previous_sibling.span.span.span.text)
    except:
        pass

Super Kings Innings
KKR Innings


In [42]:
count=0
for i in data['Match Link']:
    url2="https://espncricinfo.com"+i
    req=requests.get(url2)
    soup5=BeautifulSoup(req.content,'html.parser')
    match_id=i.split("/")[-1].split('.')[0]
    for i in soup5.find_all('div'):
        try:
            if i.span.span.span.text=="Match Details":
                team_name1=i.parent.div.span.span.span.text
                team_name2=i.previous_sibling.span.span.span.text
        except:
            pass
    team_name1=team_name1.replace(" Innings","")
    team_name2=team_name2.replace(" Innings","")
    match=team_name1+' Vs '+team_name2
    c=0
    for k in soup5.find_all('table')[0].tbody.find_all('tr'):
        j=k.find_all('td')
        if len(j)==8:
            c+=1
            name=j[0].text
            player_id=j[0].a['href'].split('/')[-1].split('.')[0].split('-')[-1]
            out=j[1].text
            runs=j[2].text
            balls=j[3].text
            fours=j[5].text
            sixs=j[6].text
            sr=j[7].text
            batting_data=batting_data.append({"Match ID":match_id,"Match":match,"Team":team_name1,"Position":c,
                                                "Batter Name":name,'Player ID':player_id,"Out":out,"Runs":runs,
                                                "Balls":balls,"4s":fours,"6s":sixs,"Strike Rate":sr},ignore_index=True)

    for k in soup5.find_all('table')[1].tbody.find_all('tr'):
        j=k.find_all('td')
        if len(j)==11:
            name=j[0].text
            player_id=j[0].a['href'].split('/')[-1].split('.')[0].split('-')[-1]
            overs=j[1].text
            mi=j[2].text
            runs=j[3].text
            wickets=j[4].text
            er=j[5].text
            dots=j[6].text
            fours=j[7].text
            sixs=j[8].text
            wides=j[9].text
            nb=j[10].text
            bowling_data=bowling_data.append({"Match ID":match_id,"Match":match,"Team":team_name2,'Bowler Name':name,"Player ID":player_id,
                                              'Overs':overs,'Made INs':mi,'Runs':runs,'Wickets':wickets,'Economy Rate':er,
                                              'Dots':dots,'4s':fours,'6s':sixs,'Wides':wides,'No Balls':nb},ignore_index=True)

    c=0
    for k in soup5.find_all('table')[2].tbody.find_all('tr'):
        j=k.find_all('td')
        if len(j)==8:
            c+=1
            name=j[0].text
            player_id=j[0].a['href'].split('/')[-1].split('.')[0].split('-')[-1]
            out=j[1].text
            runs=j[2].text
            balls=j[3].text
            fours=j[5].text
            sixs=j[6].text
            sr=j[7].text
            batting_data=batting_data.append({"Match ID":match_id,"Match":match,"Team":team_name2,"Position":c,
                                                "Batter Name":name,'Player ID':player_id,"Out":out,"Runs":runs,
                                                "Balls":balls,"4s":fours,"6s":sixs,"Strike Rate":sr},ignore_index=True)

    for k in soup5.find_all('table')[3].tbody.find_all('tr'):
        j=k.find_all('td')
        if len(j)==11:
            name=j[0].text
            player_id=j[0].a['href'].split('/')[-1].split('.')[0].split('-')[-1]
            overs=j[1].text
            mi=j[2].text
            runs=j[3].text
            wickets=j[4].text
            er=j[5].text
            dots=j[6].text
            fours=j[7].text
            sixs=j[8].text
            wides=j[9].text
            nb=j[10].text
            bowling_data=bowling_data.append({"Match ID":match_id,"Match":match,"Team":team_name1,'Bowler Name':name,"Player ID":player_id,
                                              'Overs':overs,'Made INs':mi,'Runs':runs,'Wickets':wickets,'Economy Rate':er,
                                              'Dots':dots,'4s':fours,'6s':sixs,'Wides':wides,'No Balls':nb},ignore_index=True)
    count+=1
    print(count,end="--")

1--2--3--4--5--6--7--8--9--10--11--12--13--14--15--16--17--18--19--20--21--22--23--24--25--26--27--28--29--30--31--32--33--34--35--36--37--38--39--40--41--42--43--44--45--46--47--48--49--50--51--52--53--54--55--56--57--58--59--60--61--62--63--64--65--66--67--68--69--70--71--72--73--74--

In [48]:
batting_data["Batter Name"]=batting_data["Batter Name"].apply(lambda x: x.replace("â€",""))
batting_data["Batter Name"]=batting_data["Batter Name"].apply(lambda x: x.replace("Â",""))

In [44]:
bowling_data.to_csv("Bowling.csv")

In [50]:
batting_data.to_csv("Batting.csv")

In [46]:
batting_data

,Match ID,Match,Team,Position,Batter Name,Player ID,Out,Runs,Balls,4s,6s,Strike Rate
0,1304047,Super Kings Vs KKR,Super Kings,1,Ruturaj Gaikwad,1060380,c Rana b Yadav,0,4,0,0,0.00
1,1304047,Super Kings Vs KKR,Super Kings,2,Devon Conway,379140,c SS Iyer b Yadav,3,8,0,0,37.50
2,1304047,Super Kings Vs KKR,Super Kings,3,Robin Uthappa,35582,st †Jackson b Varun,28,21,2,2,133.33
3,1304047,Super Kings Vs KKR,Super Kings,4,Ambati Rayudu,33141,run out (SS Iyer/Narine),15,17,1,1,88.23
4,1304047,Super Kings Vs KKR,Super Kings,5,Ravindra Jadeja (c),234675,not out,26,28,0,1,92.85
...,...,...,...,...,...,...,...,...,...,...,...,...
1171,1312200,Royals Vs Titans,Titans,1,Wriddhiman Saha †,279810,b Prasidh Krishna,5,7,1,0,71.42
1172,1312200,Royals Vs Titans,Titans,2,Shubman Gill,1070173,not out,45,43,3,1,104.65
1173,1312200,Royals Vs Titans,Titans,3,Matthew Wade,230193,c Parag b Boult,8,10,0,1,80.00
1174,1312200,Royals Vs Titans,Titans,4,Hardik Pandya (c),625371,c Jaiswal b Chahal,34,30,3,1,113.33


In [47]:
bowling_data

,Match ID,Match,Team,Bowler Name,Player ID,Overs,Made INs,Runs,Wickets,Economy Rate,Dots,4s,6s,Wides,No Balls
0,1304047,Super Kings Vs KKR,KKR,Umesh Yadav,376116,4,0,20,2,5.00,15,1,1,2,1
1,1304047,Super Kings Vs KKR,KKR,Shivam Mavi,1079848,4,0,35,0,8.75,12,3,2,1,1
2,1304047,Super Kings Vs KKR,KKR,Varun Chakravarthy,1108375,4,0,23,1,5.75,11,1,1,1,0
3,1304047,Super Kings Vs KKR,KKR,Sunil Narine,230558,4,0,15,0,3.75,10,0,0,0,0
4,1304047,Super Kings Vs KKR,KKR,Andre Russell,276298,4,0,38,1,9.50,7,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,1312200,Royals Vs Titans,Royals,Trent Boult,277912,4,1,14,1,3.50,17,1,0,2,0
853,1312200,Royals Vs Titans,Royals,Prasidh Krishna,917159,4,0,40,1,10.00,13,5,1,2,0
854,1312200,Royals Vs Titans,Royals,Yuzvendra Chahal,430246,4,0,20,1,5.00,10,1,0,0,0
855,1312200,Royals Vs Titans,Royals,Obed McCoy,906783,3.1,0,26,0,8.21,5,2,1,0,0


In [31]:
urlp="https://www.espncricinfo.com/cricketers/devon-conway-379140"
htmlp=requests.get(urlp)

In [32]:
soup2.find_all('table')[1].tbody.find_all('tr')[2].td.text

'Shivam Mavi'

In [17]:
player_data=pd.DataFrame(columns=["Player ID","Player Name","Team","Age","Country","Playing Role","Batting Style","Bowling Style"])

In [34]:
soupp=BeautifulSoup(htmlp.content,'html.parser')

In [35]:
for i in soupp.body.find_all("div"):
    try:
        if i.p.text=="Age":
            print(i.span.text)
        if i.p.text=='Batting Style':
            print(i.span.text)
        if i.p.text=="Bowling Style":
            print(i.span.text)
        if i.p.text=="Playing Role":
            print(i.span.text)
    except:
        pass

31y 219d
Left hand Bat
Right arm Medium
Wicketkeeper Batter


In [36]:
for i in soupp.body.find_all('span'):
    try:
        if i.text=="|":
            print(i.previous_sibling.text)
            break
    except:
        pass

New Zealand


In [18]:
players_list=[]
count=0
for z in data['Match Link']:
    url20="https://espncricinfo.com"+z
    req=requests.get(url20)
    soup20=BeautifulSoup(req.content,'html.parser')
    team_names=[]
    for i in soup20.find_all('div'):
        try:
            if i.span.span.span.text=="Match Details":
                team_names+=[i.parent.div.span.span.span.text]
                team_names+=[i.previous_sibling.span.span.span.text]
        except:
            pass
    team_names[0]=team_names[0].replace(" Innings","")
    team_names[1]=team_names[1].replace(" Innings","")
    team_names.append(team_names[1])
    team_names.append(team_names[0])
    c=0
    for k in soup20.find_all('table'):
        for j in k.tbody.find_all('tr'):
            if j.td.text=="Extras":
                break
            if j.td.a and j.td.a['href'].split("/")[1]=="cricketers":
                player_id=j.td.a['href'].split("/")[2].split("-")[-1]
                if player_id not in players_list:
                    players_list.append(player_id)
                    url_pl="https://espncricinfo.com"+j.td.a['href']
                    html_pl=requests.get(url_pl)
                    soup_pl=BeautifulSoup(html_pl.content,'html.parser')
                    for i in soup_pl.body.find_all("div"):
                        try:
                            if i.p.text=="Full Name":
                                name=i.span.text
                            if i.p.text=="Age":
                                age=i.span.text
                            if i.p.text=='Batting Style':
                                bat_style=i.span.text
                            if i.p.text=="Bowling Style":
                                bowl_style=i.span.text
                            if i.p.text=="Playing Role":
                                playing=i.span.text
                        except:
                            pass
                    for i in soup_pl.body.find_all('span'):
                        try:
                            if i.text=="|":
                                country=i.previous_sibling.text
                                break
                        except:
                            pass
                    player_data=player_data.append({"Player ID":player_id,"Player Name":name,"Team":team_names[c],"Age":age,"Country":country,
                                                    "Playing Role":playing,"Batting Style":bat_style,
                                                    "Bowling Style":bowl_style},ignore_index=True)
                    count+=1
                    print(count,end="--")
        c+=1

1--2--3--4--5--6--7--8--9--10--11--12--13--14--15--16--17--18--19--20--21--22--23--24--25--26--27--28--29--30--31--32--33--34--35--36--37--38--39--40--41--42--43--44--45--46--47--48--49--50--51--52--53--54--55--56--57--58--59--60--61--62--63--64--65--66--67--68--69--70--71--72--73--74--75--76--77--78--79--80--81--82--83--84--85--86--87--88--89--90--91--92--93--94--95--96--97--98--99--100--101--102--103--104--105--106--107--108--109--110--111--112--113--114--115--116--117--118--119--120--121--122--123--124--125--126--127--128--129--130--131--132--133--134--135--136--137--138--139--140--141--142--143--144--145--146--147--148--149--150--151--152--153--154--155--156--157--158--159--160--161--162--163--164--165--166--167--168--169--170--171--172--173--174--175--176--177--178--179--180--181--182--183--184--185--186--187--188--189--190--

In [19]:
player_data

,Player ID,Player Name,Team,Age,Country,Playing Role,Batting Style,Bowling Style
0,1060380,Ruturaj Dashrat Gaikwad,Super Kings,26y 12d,India,Batter,Right hand Bat,Right arm Offbreak
1,379140,Devon Philip Conway,Super Kings,31y 219d,New Zealand,Wicketkeeper Batter,Left hand Bat,Right arm Medium
2,35582,Robin Venu Uthappa,Super Kings,37y 93d,India,Batter,Right hand Bat,Right arm Medium
3,33141,Ambati Thirupathi Rayudu,Super Kings,37y 142d,India,Middle order Batter,Right hand Bat,Right arm Offbreak
4,234675,Ravindrasinh Anirudhsinh Jadeja,Super Kings,34y 68d,India,Allrounder,Left hand Bat,Slow Left arm Orthodox
...,...,...,...,...,...,...,...,...
185,1081442,Mayank Markande,Mumbai,25y 93d,India,Bowler,Right hand Bat,Legbreak
186,1081183,Abhijeet Tomar,KKR,27y 335d,India,Opening Batter,Right hand Bat,Right arm Offbreak
187,326017,Siddarth Kaul,RCB,32y 269d,India,Bowler,Right hand Bat,Right arm Medium
188,956871,Prerak Nileshkumar Mankad,Punjab Kings,28y 295d,India,Allrounder,Right hand Bat,Right arm Medium


In [20]:
player_data.to_csv("Player_Data.csv")